<a href="https://colab.research.google.com/github/aya-bani/changing-optimization-function/blob/main/sgmd_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np, pandas as pd, tensorflow as tf, random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def set_seeds(seed=42):
    np.random.seed(seed); random.seed(seed); tf.random.set_seed(seed)

In [24]:
data = pd.read_csv("/content/drive/MyDrive/projet mit/data.csv", encoding="latin1")
X = data[["Tf_in","Ua","Uw"]].values
y = data["Y"].values.reshape(-1,1)

scaler_X = MinMaxScaler()
Xs = scaler_X.fit_transform(X)
scaler_y = MinMaxScaler(feature_range=(0.2, 0.8))
ys = scaler_y.fit_transform(y)

In [26]:

X_train, X_temp, y_train, y_temp = train_test_split(Xs, ys, test_size=0.30, random_state=42)
X_val,   X_test, y_val,   y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

In [27]:
def build_model_final(layers=[10,5], hidden_act="sigmoid", out_act="sigmoid", lr=0.01):
    model = Sequential()
    model.add(Input(shape=(3,)))

    # Hidden layers
    model.add(Dense(layers[0], activation=hidden_act))
    for n in layers[1:]:
        model.add(Dense(n, activation=hidden_act))

    # Output layer
    model.add(Dense(1, activation=out_act))

    # Fixed optimizer (Adam, lr=0.01)
    opt = Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss="mse", metrics=["mae"])
    return model


In [28]:
def evaluate_model(model, X, y, scaler_y):
    # Predict
    y_pred = model.predict(X, verbose=0)

    # Inverse transform back to original scale
    y_true = scaler_y.inverse_transform(y)
    y_pred_inv = scaler_y.inverse_transform(y_pred)

    # Return metrics
    return (
        mean_squared_error(y_true, y_pred_inv),   # MSE
        mean_absolute_error(y_true, y_pred_inv),  # MAE
        r2_score(y_true, y_pred_inv)              # R²
    )


In [29]:
def sweep_optims(layers=[10,5], acts="sigmoid"):
    results = []
    for opt in ["adam"]:   # keep only the best optimizer
        # finer learning rates around 0.01
        for lr in [0.005, 0.007, 0.01, 0.02]:
            set_seeds(42); K.clear_session()
            model = build_model_opt(layers, hidden_act=acts, out_act="sigmoid", optimizer=opt, lr=lr)

            es = EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=True)
            hist = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=1000, batch_size=8, verbose=0,
                callbacks=[es]
            )

            # Pass scaler_y explicitly
            tr = evaluate_model(model, X_train, y_train, scaler_y)
            va = evaluate_model(model, X_val, y_val, scaler_y)
            te = evaluate_model(model, X_test, y_test, scaler_y)

            results.append({
                "optimizer": opt,
                "lr": lr,
                "train_r2": tr[2], "val_r2": va[2], "test_r2": te[2],
                "val_mse": va[0], "test_mse": te[0],
                "val_mae": va[1], "test_mae": te[1],
                "epochs_used": len(hist.history["loss"])
            })
    return pd.DataFrame(results)


In [31]:
# Run sweep
opt_results = sweep_optims(layers=[10,5], acts="sigmoid")

# Sort by R² (descending)
sorted_r2 = opt_results.sort_values("test_r2", ascending=False)

print("\n=== Full Results (sorted by Test R²) ===")
print(sorted_r2.to_string(index=False, float_format="%.4f"))

# Best configs per metric
best_r2  = opt_results.loc[opt_results["test_r2"].idxmax()]
best_mse = opt_results.loc[opt_results["test_mse"].idxmin()]
best_mae = opt_results.loc[opt_results["test_mae"].idxmin()]

print("\n=== Best Configs ===")
print("Best R² : ", best_r2.to_dict())
print("Best MSE: ", best_mse.to_dict())
print("Best MAE: ", best_mae.to_dict())

# (Optional) Save to CSV for analysis later
opt_results.to_csv("optimizer_lr_results.csv", index=False)



=== Full Results (sorted by Test R²) ===
optimizer     lr  train_r2  val_r2  test_r2  val_mse  test_mse  val_mae  test_mae  epochs_used
     adam 0.0200    0.8921  0.5980   0.8980   0.0118    0.0032   0.0752    0.0485          183
     adam 0.0100    0.8930  0.5958   0.8929   0.0119    0.0034   0.0752    0.0500          304
     adam 0.0070    0.8933  0.5976   0.8897   0.0118    0.0035   0.0750    0.0507          414
     adam 0.0050    0.8938  0.5989   0.8864   0.0118    0.0036   0.0750    0.0513          566

=== Best Configs ===
Best R² :  {'optimizer': 'adam', 'lr': 0.02, 'train_r2': 0.892145724458597, 'val_r2': 0.5980032201365011, 'test_r2': 0.8980325155068367, 'val_mse': 0.011786760402627024, 'test_mse': 0.003195705176479718, 'val_mae': 0.0751653652071953, 'test_mae': 0.04845859555602068, 'epochs_used': 183}
Best MSE:  {'optimizer': 'adam', 'lr': 0.02, 'train_r2': 0.892145724458597, 'val_r2': 0.5980032201365011, 'test_r2': 0.8980325155068367, 'val_mse': 0.011786760402627024, 'te